# Imports

In [ ]:
import sys

%cd Cap_Repo
    
from PIL import Image
import requests
import torch
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode
from models.blip import blip_decoder

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
image_size = 480
caption_model_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_capfilt_large.pth'
vqa_model_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_vqa_capfilt_large.pth'    
    
model_caption = blip_decoder(pretrained=caption_model_url, image_size=image_size, vit='base')
model_caption.eval()
model_caption = model_caption.to(device)

model_question = blip_vqa(pretrained=vqa_model_url, image_size=image_size, vit='base')
model_question.eval()
model_question = model_question.to(device)

# Functions

In [ ]:
def load_image(image_size, device, img_url): #resize
    raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')   

    w,h = raw_image.size
    display(raw_image.resize((w//5,h//5)))
    
    transform = transforms.Compose([
        transforms.Resize((image_size,image_size),interpolation=InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
        ]) 
    image = transform(raw_image).unsqueeze(0).to(device)   
    return image

def generate_caption(img_url):
  image = load_image(image_size=image_size, device=device, img_url=imge_url)
  with torch.no_grad():
  caption = model_caption.generate(image, sample=False, num_beams=3, max_length=20, min_length=5) #input = image, sampling = true/false, beams = ?, max no of words in caption, min no of words in caption
  print('caption: '+caption[0])

def generate_answer(img_url, question): # eg. 'where is the woman sitting?'
  image = load_image(image_size=image_size, device=device, img_url=imge_url)
  with torch.no_grad():
  answer = model_question(image_1, question, train=False, inference='generate') 
  print('answer: '+answer[0])



In [ ]:
generate_caption('') #put url of image here
generate_answer('', '') #put url of image here and question related to the picture